In [3]:
import pandas as pd
import time
from concurrent.futures import ProcessPoolExecutor
# import dask.dataframe as dd



In [4]:
candidate_path = '/home/it/Desktop/RUN/Official/Source/CSAB_SuperNumerary/filtered_candidates.csv'
choice_path = '/home/it/Desktop/RUN/Official/Source/CSAB_SuperNumerary/Choice.csv'
seats_path = '/home/it/Desktop/RUN/Official/Source/CSAB_SuperNumerary/Seats.csv'

Inst_path = '/home/it/Desktop/RUN/Official/Source/CSAB_SuperNumerary/Institutes.csv'
merge_path='candidate_choice_merge.csv'
normal_path='Normal_Virtualized_Choices.csv'
ds_path='DS_Virtualized_Choices.csv'

candidate = pd.read_csv(candidate_path)
candidate = candidate.rename(columns={'CAT': 'Cat', 'PwD': 'SubCat', 'SCode': 'StateCode'})
choice = pd.read_csv(choice_path)
choice = choice.rename(columns={'Rollno': 'RollNo', 'Instcd': 'InstCode', 'Brcd': 'Brcode', 'Optno':'ChoiceNo'})

# candidate = dd.read_csv(candidate_path)
# choice = dd.read_csv(choice_path)
#updated seats
seat = pd.read_csv(seats_path)
seat = seat.rename(columns={'InstCd': 'InstCode', 'BrCd': 'Brcode','StCd1':'Stcode1','StCd2':'Stcode2','StCd2':'Stcode2','StCd3':'Stcode3','StCd1':'Stcode1','StCd4':'Stcode4','StCd5':'Stcode5','StCd6':'Stcode6','StCd7':'Stcode7','StCd8':'Stcode8','StCd9':'Stcode9','StCd10':'Stcode10','TSeat':'TotalSeat'})



In [ ]:
start_ti = time.time()
df = pd.merge(candidate, choice, on="RollNo")
df.to_csv(merge_path, index=False)
end_ti = time.time()
print(f"time taken to merge candidate and choice and save as csv- {end_ti- start_ti}")

In [6]:
institutes={}
for index, row in seat.iterrows():
    if row['Quota']=='OS' or row['Quota']=='HS':
        if not row['InstCode'] in institutes.keys():
            institutes[row['InstCode']]=[row['Stcode1'],row['Stcode2'],row['Stcode3'],row['Stcode4'],row['Stcode5'],row['Stcode6'],row['Stcode7'],row['Stcode8'],row['Stcode9'],row['Stcode10']]

In [7]:
Inst = pd.read_csv(Inst_path)
Quotas={}
for index, row in Inst.iterrows():
        Quotas[row['InstCd']]=row['SeatType']

In [8]:
del df, candidate, choice, seat

In [ ]:
def determine_virtual_categories(row):
    if row['Cat'] == 'BC':
        return ['OP', 'BC']
    elif row['Cat'] == 'GN':
        return ['OP']
    elif row['Cat'] == 'EW':
        return ['OP', 'EW']
    elif row['Cat'] == 'SC':
        return ['OP', 'SC']
    elif row['Cat'] == 'ST':
        return ['OP', 'ST']
    else:
        return []

def determine_virtual_sub_categories(row):
    if row['SubCat'] == 2:
        return ['NO']
    elif row['SubCat'] == 1:
        return ['NO', 'PH']
    else:
        return []

def determine_virtual_gender(row):
    if row['Gender'] == 2:
        return ['F', 'B']
    else:
        return ['B']

# def determine_virtual_state_quota(row):
#     if Quotas[row['InstCode']] == 'AI':
#         return ['AI']
#     if row['InstCode'] == 209 and row['StateCode']=='GO':
#         return ['GO']    
    
#     if row['InstCode'] == 225 and row['StateCode']=='JK':
#         return ['JK']    
    
#     if row['InstCode'] == 225 and row['StateCode']=='LA':
#         return ['LA']    
#     state_codes="ABCD"
#     if row['InstCode']  in institutes.keys():
#         state_codes = institutes[row['InstCode']]
#     if row['StateCode'] in state_codes:
#         if Quotas[row['InstCode']] == 'OH':
#             return ['HS']
#         else:
#             return ['AI', 'HS']
#     else:
#         if Quotas[row['InstCode']] == 'OH':
#             return ['OS']
#         else:
#             return ['AI']

def process_chunk(chunk):
    chunk['Cat_V'] = chunk.apply(determine_virtual_categories, axis=1)
    chunk = chunk.explode('Cat_V')
    chunk['SubCat_V'] = chunk.apply(determine_virtual_sub_categories, axis=1)
    chunk = chunk.explode('SubCat_V')
    chunk['Gender_V'] = chunk.apply(determine_virtual_gender, axis=1)
    chunk = chunk.explode('Gender_V')
    # chunk['State_quota_V'] = chunk.apply(determine_virtual_state_quota, axis=1)
    # chunk = chunk.explode('State_quota_V')
    return chunk

start_time_loading_merging = time.time()

chunks = pd.read_csv(merge_path, chunksize=100000)

processed_chunks = []

with ProcessPoolExecutor() as executor:
    for processed_chunk in executor.map(process_chunk, chunks):
        processed_chunks.append(processed_chunk)

merged_df = pd.concat(processed_chunks, ignore_index=True)

end_time_loading_merging = time.time()
print(f"Time taken for loading tables and merging and running virtulization: {end_time_loading_merging - start_time_loading_merging:.2f} seconds")


print(f"Now Sorting:")
merged_df.sort_values(by=['RollNo', 'ChoiceNo'], inplace=True)
end_time_sorting = time.time()

print(f"Time taken for Sorting: {end_time_sorting - end_time_loading_merging:.2f} seconds")


print(f"Now creating Virtual count")
merged_df['ChoiceNo_V'] = merged_df.groupby(['RollNo']).cumcount() + 1
end_time_V = time.time()
print(f"Time taken for creating Virtual count: {end_time_V- end_time_sorting:.2f} seconds")


# # Ensure Adv_DS_V column is empty initially
# merged_df['Adv_DS_V'] = None

# # Identify RollNos with Adv_DS == 1.0
# adv_ds_rollnos = merged_df[merged_df['Adv_DS'] == 1.0]['RollNo'].unique()

# ds_df = merged_df[merged_df['RollNo'].isin(adv_ds_rollnos)]
# normal_df = merged_df[~merged_df['RollNo'].isin(adv_ds_rollnos)]

# # Create new rows for DS RollNos at the end of each choice
# new_rows = []
# for rollno in adv_ds_rollnos:
#     unique_choices = ds_df[ds_df['RollNo'] == rollno]['ChoiceNo'].unique()
#     for choice in unique_choices:
#         choice_rows = ds_df[(ds_df['RollNo'] == rollno) & (ds_df['ChoiceNo'] == choice)]
#         if choice_rows.iloc[0]['InstCode'] < 200:  # Only add DS rows for IITs
#             last_virtual_choice_no = choice_rows['ChoiceNo_V'].max() + 1
#             last_row = choice_rows.iloc[-1]
#             new_row = last_row.copy()
#             new_row['ChoiceNo_V'] = last_virtual_choice_no
#             new_row['Adv_DS_V'] = 1
#             new_rows.append(new_row)

# new_rows_df = pd.DataFrame(new_rows)

# # Append new rows to the DS DataFrame
# ds_final_df = pd.concat([ds_df, new_rows_df], ignore_index=True)

# # Recalculate VirtualChoiceNumber to account for new DS rows
# ds_final_df.sort_values(by=['RollNo', 'ChoiceNo'], inplace=True)
# ds_final_df['ChoiceNo_V'] = ds_final_df.groupby(['RollNo']).cumcount() + 1

# Select columns to reorder them as needed
reordered_columns = ['RollNo', 'InstCode', 'Brcode', 'ChoiceNo', 'ChoiceNo_V', 'Cat_V', 'SubCat_V', 'Gender_V', 'State_quota_V', 'Validity', 'Adv_DS_V', 'Nationality']

normal_df = merged_df[reordered_columns]
# ds_final_df = ds_final_df[reordered_columns]

print(f"Now creating csv file")
normal_df.to_csv(normal_path, index=False)
# ds_final_df.to_csv(ds_path, index=False)


print(f"Time taken for creating csv file: {time.time() - end_time_V:.2f} seconds")